# Układy równań liniowych

### Pojęcia warte poznania

* Układ równań liniowych: https://pl.wikipedia.org/wiki/Układ_równań_liniowych
* Rząd macierzy: https://pl.wikipedia.org/wiki/Rząd_macierzy
* Kombinacja liniowa: https://pl.wikipedia.org/wiki/Kombinacja_liniowa
* Eliminacja Gaussa: https://pl.wikipedia.org/wiki/Metoda_eliminacji_Gaussa, Kincaid-Cheney_*_ str. 245, pełny pseudokod: str. 252
* Pivoting: https://en.wikipedia.org/wiki/Pivot_element#Partial_and_complete_pivoting, K.C. str. 261, pełny pseudokod: str. 267
* Norma wektora: https://pl.wikipedia.org/wiki/Przestrze%C5%84_unormowana, K.C. str. 320
* Norma macierzy: https://pl.wikipedia.org/wiki/Norma_macierzowa
* Macierz dodatnio określona: https://pl.wikipedia.org/wiki/Określoność_formy
* Faktoryzacja LU: https://pl.wikipedia.org/wiki/Metoda_LU, K.C. str. 294
* Faktoryzacja Cholesky'ego: https://pl.wikipedia.org/wiki/Rozkład_Choleskiego, K.C. str. 305

Dodatkowo:
* Wskaźnik uwarunkowania: https://pl.wikipedia.org/wiki/Wska%C5%BAnik_uwarunkowania, K.C. str.321
* Metoda Jacobiego: https://en.wikipedia.org/wiki/Jacobi_method, K.C. 323

Książka dla wytrwałych (naprawdę): Y. Saad "Iterative Methods for Sparse Linear Systems"

In [1]:
from typing import Optional, Tuple
import numpy as np

### Zadanie rozgrzewkowe:
Napisać mnożenie macierzy w ulubionym_\**_ języku programowania.

**Pytanko:** jakie muszą być wymiary mnożonych macierzy? (Który wymiar musi się "zgadzać"?)

**Zadanko:** Uzupełnić brakujące wymiary macierzy w docstringu (z dokładnością do ["alfa-konwersji"](https://pl.wikipedia.org/wiki/Konwersja_α))

In [2]:
def agh_superfast_matrix_multiply(a: np.matrix, b: np.matrix) -> np.matrix:
    """Perform totally ordinary multiplication of matrices.
    
    :param a: matrix with dimensions _ by _
    :param b: matrix with dimensions _ by _
    :return:  matrix with dimensions _ by _
    """
    n = a.shape[0]
    m = a.shape[1]
    p = b.shape[1]
    c = np.zeros((n, p))
    
    for i in range(0, n):
        for j in range(0, p):
            for k in range(0, m):
                c[i, j] += a[i, k] * b[k, j]
                
    return c

m1 = np.matrix([[1, 2],
                [3, 4],
                [4, 5],
                [5, 1]])

m2 = np.matrix([[1, 2, 3],
                [4, 5, 6]])

res = agh_superfast_matrix_multiply(m1, m2)
assert np.allclose(res, m1 * m2), "Wrong multiplication result"

### Zadania
1. Przeczytać rozdz. 7. Kincaida i Cheney'a (Systems of Linear Equations).
2. Przeczytać rozdz. 8. Kincaida i Cheney'a (Additional Topics Concerning Systems of Linear Equations).
3. Napisać kod (w ulubionym_\**_ języku) do eliminacji Gaussa z i bez pivotingu.
4. Rozwiązać poniższy układ równań z pivotingiem i bez, porównać wyniki:

In [3]:
A = np.matrix([[0.0001, -5.0300, 5.8090, 7.8320],
               [2.2660, 1.9950,  1.2120, 8.0080],
               [8.8500, 5.6810,  4.5520, 1.3020],
               [6.7750, -2.253,  2.9080, 3.9700]])

b = np.matrix([9.5740, 7.2190, 5.7300, 6.2910]).transpose()

x = np.linalg.solve(A, b)

In [6]:
#Zadanie 3
def gauss_no_pivot(a, b):
    n = a.shape[0]
    x = np.zeros(n)
    for i in range(0, n):
        for j in range(i+1, n):
            x_mult = a[j, i] / a[i, i]
            a[j, i] = x_mult
            for k in range(i+1, n):
                a[j, k] = a[j, k] - x_mult * a[i, k]
            b[j] = b[j] - x_mult * b[i]
    x[n - 1] = b[n - 1] / a[n - 1, n - 1]
    for i in range(n - 1, -1, -1):
        sum = b[i]
        for j in range(i + 1, n):
            sum = sum - a[i, j] * x[j]
        x[i] = sum / a[i, i]
    return np.matrix(x).transpose()

In [7]:
x1 = np.linalg.solve(A, b)
x2 = gauss_no_pivot(A, b)
assert np.allclose(x1, x2), "Wrong result"

In [12]:
#Zadanie 4
def gauss_pivot(a, b):
    n = a.shape[0]
    x = np.zeros(n)
    s = np.zeros(n)
    d = np.zeros(n, dtype=np.int)
    
    for i in range(0, n):
        d[i] = i
        smax = 0
        for j in range(0, n):
            smax = max(smax, abs(a[i, j]))
        s[i] = smax
        
    for k in range(0, n - 1):
        rmax = 0
        for i in range(k,n):
            r = abs(a[d[i], k] / s[d[i]])
            if r > rmax:
                rmax = r
                j = i
        tmp = d[j]
        d[j] = d[k]
        d[k] = tmp
        for i in range(k + 1,n):
            xmult = a[d[i], k] / a[d[k], k]
            a[d[i],k] = xmult
            for j in range(k+1, n):
                a[d[i], j] = a[d[i], j] - xmult * a[d[k], j]
    
    for k in range(0,n):
        for i in range(k+1,n):
            b[d[i]] = b[d[i]] - a[d[i], k] * b[d[k]]
    m = n-1
    x[m] = b[d[m]] / a[d[m], m]
    for i in range(n-1, -1, -1):
        sum = b[d[i]]
        for j in range(i+1, n):
            sum = sum - a[d[i], j] * x[j]
        x[i] = sum / a[d[i], i]
    return np.matrix(x).transpose()
    
    

In [13]:
x1 = np.linalg.solve(A, b)
x2 = gauss_pivot(A, b)
assert np.allclose(x1, x2), "Wrong result"

**Pytanie**: dlaczego wołamy `transpose()` na wektorze `b`?

Sprawdźmy, czy rozwiązanie jest ok (**Pytanie'**: dlaczego po prostu nie użyjemy `==` lub jakiegoś `equals`?):

In [14]:
np.allclose(np.dot(A, x), b)

False

### Zadania, c.d.

5. Zaimplementować algorytm faktoryzacji LU macierzy
6. (*) Zaimplementować funkcję sprawdzającą, czy dana macierz jest symetryczna i dodatnio określona
7. Zaimplementować algorytm faktoryzacji Cholesky'ego macierzy

In [15]:
def agh_superfast_lu(a: np.matrix) -> Optional[Tuple[np.matrix, np.matrix]]:
    """Perform LU decomposition of a matrix.
    
    :param a: _
    :return:  _
    """
    n = a.shape[0]
    l = np.zeros((n,n))
    u = np.zeros((n,n))
    
    for k in range(0, n):
        l[k,k] = 1
        for j in range(k, n):
            sum = 0
            for s in range(0,k):
                sum += l[k,s]*u[s,j]
            u[k,j] = a[k,j]-sum
        for i in range(k+1, n):
            sum = 0
            for s in range(0, k):
                sum += l[i,s]*u[s,k]
            l[i,k] = (a[i,k]-sum)/u[k,k]
    return (l, u)

def agh_superfast_check_spd(a: np.matrix) -> bool:
    """Check whether a matrix is symmetric and positive-definite (SPD).
    
    :param a: _
    """
    l = agh_superfast_cholesky(a)
    llt = agh_superfast_matrix_multiply(l, l.transpose())
    return np.allclose(llt, a)

def agh_superfast_cholesky(a: np.matrix) -> Optional[np.matrix]:
    """Perform a Cholesky decomposition of a matrix.
    
    :param a: _
    :return:  _
    """
    n = a.shape[0]
    l = np.zeros((n,n))
    
    for k in range(0, n):
        sum = 0
        for s in range(0, k):
            sum += l[k,s]*l[k,s]
        l[k,k] = pow(a[k,k]-sum, 1/2)
        for i in range(k+1, n):
            sum = 0
            for s in range(0, k):
                sum += l[i,s]*l[k,s]
            l[i,k] = (a[i,k]-sum)/l[k,k]
    return l

### Zadania, opcjonalnie
5. zaimplementować metodę Jacobiego (iteracyjne rozwiązywanie układu równań liniowych)
6. za pomocą tejże metody rozwiązać powyższy układ równań

\*  wszystkie referencje odnoszą się do [książki](https://wiki.iiet.pl/lib/exe/fetch.php?media=studia:przedmioty:mownit:numerical_mathematics_and_computing.pdf) David Kincaid, Ward Cheney - "Numerical Mathematics and Computing, 6th edition"
\** _ulubiony_ język programowania staramy się pojmować rozsądnie, tj. z wyłączeniem języków pokroju Prologa, języków z [tej listy](https://en.wikipedia.org/wiki/Esoteric_programming_language) oraz Assemblera i PHP. Haskella można używać na własną odpowiedzialność.